# Keras exercise

In this exercise you will be creating a Keras model by loading a data set, preprocessing input data, building a Sequential Keras model and compiling the model with a training configuration. Afterwards, you train your model on the training data and evaluate it on the test set. To finish this exercise, you will past the accuracy of your model to the Coursera grader.

This notebook is tested in IBM Watson Studio under python 3.6

## Data

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

## Instructions

We start by installing and importing everything we need for this exercise:

In [ ]:
!pip install tensorflow==2.2.0

     ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 179.7/516.3 MB 7.9 MB/s eta 0:00:43

In [ ]:
import tensorflow as tf
if not tf.__version__ == '2.2.0-rc0':
    print(tf.__version__)
    raise ValueError('please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)')

2022-02-03 13:57:31.652639: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-03 13:57:31.652779: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.7.0


ValueError: please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)

IMPORTANT! => Please restart the kernel by clicking on "Kernel"->"Restart and Clear Outout" and wait until all output disapears. Then your changes are beeing picked up

As you can see, we use Keras' Sequential model with only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set:

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
seed = 1337
np.random.seed(seed)
from tensorflow.keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

2121728/2110848 [==============================] - 0s 0us/step


Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [4]:
x_train
print("original max length: ", max([len(x) for x in x_train]), "and min length: ", min([len(x) for x in x_train]))


original max length:  2376 and min length:  2


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [7]:
print("tokenized max length: ", max([len(x) for x in x_train]), "and min length: ", min([len(x) for x in x_train]))


tokenized max length:  1000 and min length:  1000


In [13]:
y_train.shape

(8982, 46)

## 1. Exercise part: label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [9]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [10]:
x_train.shape

(8982, 1000)

## 2. Exercise part: model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [16]:
model = Sequential()
model.add(Dense(512, activation = "relu", input_shape = (x_train.shape[-1],)))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[-1], activation = "softmax"))

## 3. Exercise part: model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [17]:
model.compile(loss = "categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

## 4. Exercise part: model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='loss', patience=4)
batch_size = 32
model.fit(x_train, y_train, epochs = 15, batch_size = batch_size, callbacks = [es], validation_split=0.2)
score = model.evaluate(x_test, y_test, batch_size = batch_size)

Epoch 1/15
225/225 [==============================] - 4s 12ms/step - loss: 1.3490 - accuracy: 0.6932 - val_loss: 0.9728 - val_accuracy: 0.7947
Epoch 2/15
225/225 [==============================] - 2s 11ms/step - loss: 0.6637 - accuracy: 0.8448 - val_loss: 0.8578 - val_accuracy: 0.8086
Epoch 3/15
225/225 [==============================] - 2s 11ms/step - loss: 0.4248 - accuracy: 0.9009 - val_loss: 0.8997 - val_accuracy: 0.7930
Epoch 4/15
225/225 [==============================] - 2s 10ms/step - loss: 0.3098 - accuracy: 0.9239 - val_loss: 0.8907 - val_accuracy: 0.8058
Epoch 5/15
225/225 [==============================] - 2s 11ms/step - loss: 0.2427 - accuracy: 0.9395 - val_loss: 0.9269 - val_accuracy: 0.7991
Epoch 6/15
225/225 [==============================] - 2s 11ms/step - loss: 0.2135 - accuracy: 0.9434 - val_loss: 0.9688 - val_accuracy: 0.7974
Epoch 7/15
225/225 [==============================] - 2s 10ms/step - loss: 0.1864 - accuracy: 0.9502 - val_loss: 0.9852 - val_accuracy: 0.7963

If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [20]:
score[1]

0.7760462760925293

Congratulations, now it's time to submit your result to the Coursera grader by executing the following cells (Programming Assingment, Week2). 

We have to install a little library in order to submit to coursera


In [21]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2022-02-03 13:14:50--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2022-02-03 13:14:50 (30.5 MB/s) - ‘rklib.py’ saved [2540/2540]



Please provide your email address and obtain a submission token (secret) on the grader’s submission page in coursera, then execute the cell

In [22]:
from rklib import submit
import json

key = "XbAMqtjdEeepUgo7OOVwng"
part = "HCvcp"
email = "ivanbfernandes@hotmail.com"
token = "RgNUXnB5TsFpK0iE"


submit(email, token, 'XbAMqtjdEeepUgo7OOVwng', part, [part], json.dumps(score[1]*100))

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"ozVf2","id":"tE4j0qhMEeecqgpT6QjMdA~ozVf2~R8RQ5oTzEeypxAqmVMBlAw","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------
